# Code 4-82 to Code 4-86

In [1]:
import pandas as pd

We are reading the initial NER dataset here. We would need to take this through identified organizations. This will provide the set of sentences that have organizations names

In [2]:
t1 = pd.read_csv("news_summary.csv",encoding="latin1")
t1["imp_col"] = t1["headlines"] + " " + t1["text"] + " " + t1["ctext"]

t2 = t1[t1.imp_col.str.len()>=10].reset_index()
len(t2),len(t1)

(4396, 4514)

After manual tagging in "7_Filter NERs" we are importing the shortlisted words from "interest_words_tag.csv" here

In [3]:
df_all_short_list = pd.read_csv("interest_words_tag.csv",header=None)
df_all_short_list.columns = ["words_interest"]
list_short = list(df_all_short_list["words_interest"].unique())


Scanning t2 to the shortlisted organizations

In [4]:
list_interest_all = []
for i in t2["imp_col"].str.lower():
    wrd_add = ''
    for j in list_short:
        if(i.find(j)>=0):
            wrd_add = j
            break;
    list_interest_all.append(wrd_add)

In [5]:
t2["short_list"] = list_interest_all
t3 = t2[t2.short_list.str.len()>=3]
len(t3)

296

t3 is the dataframe that contains "news articles" on shortlisted organizations

In [7]:
len(t2)

4396

We now get the sentiment of the sentences. For a quick illustration, we will be using Vader Sentiment package but this can be done using any sentiment technique in chapter 3



In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def get_vader_score(sents):
    score = analyser.polarity_scores(sents)
    return score['compound']

t3["score"] = t3["imp_col"].apply(get_vader_score)

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
sent_agg = t3.groupby(['short_list'], as_index = False)['score'].mean()
sent_agg

,short_list,score
0,air india,-0.123904
1,airtel,0.964140
2,bharti airtel,0.284985
3,cbfc,-0.197989
4,change.org,-0.776450
5,emami,0.959300
6,facebook,0.000423
7,foxx,0.029850
8,gandhi institute of medical,-0.092130
9,jet airways,-0.851844


The above table indicates the sentiment associated with each organization in the overall news corpus